In [1]:
import sys
import warnings
sys.path.append("../Pythonscripts")
sys.path.append(r"C:\Users\atedi\OneDrive\Documents\SageBionetworks\synapse\Scripts")
import pandas as pd
from pandas.io.json import json_normalize
import numpy as np
import synapseclient as sc
from pdkit_features import pdkit_pipeline
from myutils import get_synapse_table, get_acceleration_ts, open_filepath
from spectral_flatness import get_spectrum
import matplotlib.pyplot as plt
import seaborn as sns
import json
import pdkit
%matplotlib inline
warnings.simplefilter("ignore")

In [2]:
syn = sc.login()

Welcome, aryton tediarjo!



INFO:synapseclient_default:Welcome, aryton tediarjo!



In [3]:

def clean_accelerometer_data(data):
    data = data.dropna(subset = ["x", "y", "z"])
    date_series = pd.to_datetime(data["timestamp"], unit = "s")
    data["td"] = date_series - date_series.iloc[0]
    data["td"] = data["td"].apply(lambda x: x.total_seconds())
    data["time"] = data["td"]
    data = data.set_index("time")
    data.index = pd.to_datetime(data.index, unit = "s")
    data["AA"] = np.sqrt(data["x"]**2 + data["y"]**2 + data["z"]**2)
    return data

## function to retrieve sensors ## 
def get_sensor_types(filepath):
    data = open_filepath(filepath)
    if "sensorType" in data.columns:
        return data["sensorType"].dropna().unique()
    else:
        return np.NaN

## function to retrieve unit of measurements ##
def get_units(filepath):
    data = open_filepath(filepath)
    if "sensorType" in data.columns:
        return data["unit"].dropna().unique()
    else:
        return np.NaN

In [4]:
data = get_synapse_table(syn, ["ae8fe177-8e0f-4cd7-a9eb-bd5a82ac02d3", 
                               "4b1faed8-9f46-457d-b3cc-e158c2af4f65",
                              "86aaa92b-e31e-44ac-b1bc-3c68bd118652"], "syn12514611")

INFO:synapseclient_default:Downloading 0 files, 4 cached locally


In [5]:
data

,recordId,phoneInfo,createdOn,healthCode,walk_motion.json_path_id,balance_motion.json_path_id,walk_motion.json_pathfile,balance_motion.json_pathfile
0,526e2401-1ba8-45d9-82b7-a2f9ac0b9fa6,iPhone Simulator; iOS/12.1,2018-11-23 22:21:53.998,4b1faed8-9f46-457d-b3cc-e158c2af4f65,34191272.0,34191273.0,c:/users/atedi/.synapsecache/272/34191272/526e...,c:/users/atedi/.synapsecache/273/34191273/526e...
1,d2a9b58c-27af-490a-bf94-c634c907800e,iPhone Simulator; iOS/11.3,2018-10-23 03:49:52.674,ae8fe177-8e0f-4cd7-a9eb-bd5a82ac02d3,33628972.0,33628974.0,c:/users/atedi/.synapsecache/972/33628972/d2a9...,c:/users/atedi/.synapsecache/974/33628974/d2a9...


In [10]:
data

,stepPath,timestamp,timestampDate,uptime
0,WalkAndBalance/WalkAndBalance/walk/motion,164239.439972,2018-11-23T17:15:36.224-05:00,453850.152858


In [17]:
data

,recordId,phoneInfo,createdOn,healthCode,walk_motion.json_path_id,balance_motion.json_path_id,walk_motion.json_pathfile,balance_motion.json_pathfile,walk_motion.json_features_x,walk_motion.json_features_y,walk_motion.json_features_z,walk_motion.json_features_AA
0,8a217ec7-3b92-4771-9213-c937fdebfd6e,iPhone 7+; iOS/12.1.2,2019-02-10 17:25:58.714,0028d250-393f-4fa7-b203-0d8ec7c5137a,36770563.0,36770566.0,c:/users/atedi/.synapsecache/563/36770563/8a21...,c:/users/atedi/.synapsecache/566/36770566/8a21...,"{'no_of_steps': 24, 'mean_freeze_index': 1.228...","{'no_of_steps': 40, 'mean_freeze_index': 3.438...","{'no_of_steps': 34, 'mean_freeze_index': 1.250...","{'no_of_steps': 16, 'mean_freeze_index': 0.552..."
1,807a43f8-1575-4bd3-972f-0901e010fb93,iPhone 7+; iOS/12.1.2,2019-02-11 14:41:07.121,0028d250-393f-4fa7-b203-0d8ec7c5137a,36845206.0,36845207.0,c:/users/atedi/.synapsecache/206/36845206/807a...,c:/users/atedi/.synapsecache/207/36845207/807a...,"{'no_of_steps': 24, 'mean_freeze_index': 0.919...","{'no_of_steps': 19, 'mean_freeze_index': 1.587...","{'no_of_steps': 10, 'mean_freeze_index': 3.394...","{'no_of_steps': 17, 'mean_freeze_index': 0.880..."
2,274efa5b-fe86-424b-a712-5d5a27f16c98,iPhone 7+; iOS/12.1.2,2019-02-13 13:23:45.952,0028d250-393f-4fa7-b203-0d8ec7c5137a,36845258.0,36845261.0,c:/users/atedi/.synapsecache/258/36845258/274e...,c:/users/atedi/.synapsecache/261/36845261/274e...,"{'no_of_steps': 27, 'mean_freeze_index': 1.232...","{'no_of_steps': 28, 'mean_freeze_index': 3.046...","{'no_of_steps': 7, 'mean_freeze_index': 1.4154...","{'no_of_steps': 15, 'mean_freeze_index': 0.717..."
3,aed9f4e7-d879-4fad-ba89-3674a04cdedf,iPhone 7+; iOS/12.1.4,2019-02-15 15:00:50.645,0028d250-393f-4fa7-b203-0d8ec7c5137a,36878920.0,36878921.0,c:/users/atedi/.synapsecache/920/36878920/aed9...,c:/users/atedi/.synapsecache/921/36878921/aed9...,"{'no_of_steps': 31, 'mean_freeze_index': 1.645...","{'no_of_steps': 15, 'mean_freeze_index': 4.321...","{'no_of_steps': 0, 'mean_freeze_index': 1.3357...","{'no_of_steps': 16, 'mean_freeze_index': 0.516..."
4,1b3042e3-fed5-4dbb-8ca2-be4e55e6c9e8,iPhone 7+; iOS/12.1.4,2019-02-18 21:39:19.616,0028d250-393f-4fa7-b203-0d8ec7c5137a,36932400.0,36932404.0,c:/users/atedi/.synapsecache/400/36932400/1b30...,c:/users/atedi/.synapsecache/404/36932404/1b30...,"{'no_of_steps': 24, 'mean_freeze_index': 1.753...","{'no_of_steps': 35, 'mean_freeze_index': 3.224...","{'no_of_steps': 24, 'mean_freeze_index': 1.737...","{'no_of_steps': 13, 'mean_freeze_index': 0.534..."
5,783dcc46-daa9-4db0-b37e-912d3cac9afe,iPhone 7+; iOS/12.1.4,2019-02-22 14:00:54.695,0028d250-393f-4fa7-b203-0d8ec7c5137a,37013365.0,37013367.0,c:/users/atedi/.synapsecache/365/37013365/783d...,c:/users/atedi/.synapsecache/367/37013367/783d...,"{'no_of_steps': 20, 'mean_freeze_index': 0.582...","{'no_of_steps': 36, 'mean_freeze_index': 8.474...","{'no_of_steps': 17, 'mean_freeze_index': 2.181...","{'no_of_steps': 14, 'mean_freeze_index': 0.650..."
6,3497b25f-abc6-4a84-845c-fbd31ee2babc,iPhone 7+; iOS/12.1.4,2019-02-25 13:54:39.119,0028d250-393f-4fa7-b203-0d8ec7c5137a,37081135.0,37081138.0,c:/users/atedi/.synapsecache/135/37081135/3497...,c:/users/atedi/.synapsecache/138/37081138/3497...,"{'no_of_steps': 19, 'mean_freeze_index': 0.449...","{'no_of_steps': 22, 'mean_freeze_index': 6.833...","{'no_of_steps': 8, 'mean_freeze_index': 1.6563...","{'no_of_steps': 13, 'mean_freeze_index': 0.663..."
7,51ec4f2a-70f6-4d90-a09d-6f074f6e863d,iPhone 7+; iOS/12.1.4,2019-02-27 12:57:04.424,0028d250-393f-4fa7-b203-0d8ec7c5137a,37135116.0,37135120.0,c:/users/atedi/.synapsecache/116/37135116/51ec...,c:/users/atedi/.synapsecache/120/37135120/51ec...,"{'no_of_steps': 21, 'mean_freeze_index': 0.845...","{'no_of_steps': 4, 'mean_freeze_index': 5.2113...","{'no_of_steps': 5, 'mean_freeze_index': 4.2283...","{'no_of_steps': 10, 'mean_freeze_index': 0.685..."
8,a1f2fc61-2261-4978-addd-db93a950c6ba,iPhone 7+; iOS/12.1.4,2019-02-27 14:14:39.487,0028d250-393f-4fa7-b203-0d8ec7c5137a,

In [23]:
import ast
def map_to_json(params):
    params  = ast.literal_eval(params)
    if isinstance(params, dict):
        return params
    else:
        return np.NaN
    
def normalize_feature(data, feature):
    normalized_data = data[feature].map(map_to_json) \
                                .apply(pd.Series) \
                                .fillna("#ERROR").add_prefix('{}.'.format(feature))
    data = pd.concat([data, normalized_data], 
                     axis = 1).drop(feature, axis = 1)
    return data

In [24]:
data = pd.read_csv("../PythonScripts/test.csv", index_col = [0])

In [26]:
data = normalize_feature(data, "walk_motion.json_features_x")
data = normalize_feature(data, "walk_motion.json_features_y")
data = normalize_feature(data, "walk_motion.json_features_z")
data = normalize_feature(data, "walk_motion.json_features_AA")

In [28]:
data.columns

Index(['recordId', 'phoneInfo', 'createdOn', 'healthCode',
       'walk_motion.json_path_id', 'balance_motion.json_path_id',
       'walk_motion.json_pathfile', 'balance_motion.json_pathfile',
       'walk_motion.json_features_x.no_of_steps',
       'walk_motion.json_features_x.mean_freeze_index',
       'walk_motion.json_features_x.median_freeze_index',
       'walk_motion.json_features_x.max_freeze_index',
       'walk_motion.json_features_x.count_freeze_index',
       'walk_motion.json_features_x.speed_of_gait',
       'walk_motion.json_features_x.gait_step_regularity',
       'walk_motion.json_features_x.gait_stride_regularity',
       'walk_motion.json_features_x.gait_symmetry',
       'walk_motion.json_features_x.frequency_of_peaks',
       'walk_motion.json_features_y.no_of_steps',
       'walk_motion.json_features_y.mean_freeze_index',
       'walk_motion.json_features_y.median_freeze_index',
       'walk_motion.json_features_y.max_freeze_index',
       'walk_motion.json_featur

In [71]:
test.join(json_normalize(test["sensor"].map(json.loads)).drop("sensor", axis = 1))

TypeError: the JSON object must be str, bytes or bytearray, not dict

In [58]:
var = "AA"
gp = pdkit.GaitProcessor(duration=data.td[-1])
data = gp.resample_signal(data)
    ### instantiate empty dictionary ###
feature_dict = {}
try:  
    no_of_steps = gp.gait(data[var])[0]
except:
    no_of_steps = 0
try:
    gait_step_regularity, gait_stride_regularity, gait_symmetry = gp.gait_regularity_symmetry(data[var])
except:
    gait_step_regularity = 0
    gait_stride_regularity = 0
    gait_symmetry = 0
try:
    frequency_of_peaks = gp.frequency_of_peaks(data[var])
except:
    frequency_of_peaks = 0
try:
    freeze_index = gp.freeze_of_gait(data[var])[1]
except:
    freeze_index = 0
try:
    freeze_count = sum(i > 2.0 for i in freeze_index)
except:
    freeze_count = 0
try:
    speed_of_gait = gp.speed_of_gait(data[var], wavelet_level = 6)
except:
    speed_of_gait = 0
        
### fill in values to each keys
feature_dict["no_of_steps_".format(var)] = no_of_steps
feature_dict["mean_freeze_index_".format(var)] = np.mean(freeze_index)
feature_dict["median_freeze_index_".format(var)] = np.median(freeze_index)
feature_dict["max_freeze_index_".format(var)] = np.max(freeze_index)
feature_dict["count_freeze_index_".format(var)] = freeze_count
feature_dict["speed_of_gait_".format(var)] = speed_of_gait
feature_dict["gait_step_regularity_".format(var)] = gait_step_regularity
feature_dict["gait_stride_regularity_".format(var)] = gait_stride_regularity
feature_dict["gait_symmetry_".format(var)] = gait_symmetry
feature_dict["frequency_of_peaks_".format(var)] = frequency_of_peaks
        
    ## Final Check to clear nan values to zero ##
for k, v in feature_dict.items():
    if np.isnan(v):
        feature_dict[k] = 0

Sampling rate is 99.33333333333333 Hz
Sampling rate is 99.33333333333333 Hz


In [59]:
feature_dict

{'no_of_steps_': 0,
 'mean_freeze_index_': 8.2914,
 'median_freeze_index_': 3.0395563,
 'max_freeze_index_': 117.14901,
 'count_freeze_index_': 36,
 'speed_of_gait_': 0.031323065007508595,
 'gait_step_regularity_': 0,
 'gait_stride_regularity_': 0,
 'gait_symmetry_': 0,
 'frequency_of_peaks_': 0.8908512228521506}

In [56]:
data

,timestamp,uptime,x,y,z,td,AA
time,,,,,,,
1970-01-01 00:00:00.000,0.086895,7177.766841,-0.065855,-0.918915,-0.401382,0.000000,1.003714
1970-01-01 00:00:00.010,0.096919,7177.776865,-0.044920,-0.910599,-0.397156,0.010024,0.993706
1970-01-01 00:00:00.020,0.106960,7177.786906,-0.034681,-0.906677,-0.394943,0.020065,0.989043
1970-01-01 00:00:00.030,0.117011,7177.796957,-0.030088,-0.906342,-0.393860,0.030116,0.988255
1970-01-01 00:00:00.040,0.127042,7177.806988,-0.053099,-0.917908,-0.397034,0.040147,1.000581
1970-01-01 00:00:00.050,0.137073,7177.817019,-0.066343,-0.921478,-0.402405,0.050178,1.006491
1970-01-01 00:00:00.060,0.147106,7177.827052,-0.064909,-0.918045,-0.398666,0.060211,1.001793
1970-01-01 00:00:00.070,0.157138,7177.837084,-0.040281,-0.910553,-0.395752,0.070243,0.993007
1970-01-01 00:00:00.080,0.167169,7177.847115,-0.036405,-0.907639,-0.392914,0.080274,0.989140
